In [1]:
import numpy as np
import torch as torch
import torch.nn as nn
from tqdm import tqdm
import matplotlib.pyplot as plt

In [2]:
device = "cuda:1"

In [3]:
def trainer(model, gener, iters, opt, loss_fn, verbose= True, non_lin = False, device= "cuda:1"):        
    for i in tqdm(range(iters), desc = 'Progress Bar'):
        (X, Y) = gener.get_test(64)
        preds= model(torch.tensor(X, device = device, dtype= torch.float32), non_lin = non_lin)
        loss= loss_fn(preds.transpose(1, 2), torch.tensor(Y, device = device))

        loss.backward()
        opt.step()
        opt.zero_grad()


        if i%5000 == 0 and verbose:  
            with torch.no_grad():
                (X_train, Y_train) = gener.get_train(10000)
                preds_train= torch.argmax(model(torch.tensor(X_train, device = device, dtype= torch.float32), non_lin = non_lin), dim = 2)
                train_acc = (torch.sum(1*(preds_train == torch.tensor(Y_train, device = device)))/2e5).detach().cpu().numpy()

                (X_valid, Y_valid) = gener.get_test(10000)
                preds_valid= torch.argmax(model(torch.tensor(X_valid, device = device, dtype= torch.float32), non_lin = non_lin), dim = 2)
                test_acc = (torch.sum(1*(preds_valid == torch.tensor(Y_valid, device = device)))/2e5).detach().cpu().numpy()

            print("step %i: loss = %f, train_acc= %f, test_acc= %f"%(i, loss, train_acc, test_acc))
            torch.save({"model": model.state_dict(),
                        "opt": opt.state_dict(),
                        "num_train": i}, "./Sorting_limited.tar")
    print("final loss= %f"%(loss.detach().cpu().numpy()))

    
class MultiHeadAttention_wOhead(nn.Module):
    def __init__(self, seq_len, input_dim, output_dim, attn_dim, num_heads):
        super().__init__()
        embed_dim = input_dim
        assert attn_dim % num_heads == 0
        
        self.embed= nn.Linear(input_dim, embed_dim, bias = False)
        self.P = nn.Parameter(torch.randn(seq_len, embed_dim), requires_grad= True)
        self.key = nn.Linear(embed_dim, attn_dim, bias = False)
        self.query = nn.Linear(embed_dim, attn_dim, bias = False)
        self.value = nn.Linear(embed_dim, attn_dim, bias = False)
        self.proj = nn.Linear(attn_dim, output_dim, bias = False)

        self.n_head = num_heads
        self.seq_len = seq_len
        self.input_dim= input_dim
        self.attn_dim = attn_dim

    def forward(self, input, non_lin = True):
        N, S, E = input.shape
        assert (S, E) == (self.seq_len, self.input_dim), "Wrong input!"

        # X = self.embed(input) + self.P
        X = input

        Q = torch.reshape(self.query(X), (N, self.seq_len, self.n_head, self.attn_dim//self.n_head)).transpose(1 , 2)
        K = torch.reshape(self.key(X), (N, self.seq_len, self.n_head, self.attn_dim//self.n_head)).transpose(1, 2)
        V = torch.reshape(self.value(X), (N, self.seq_len, self.n_head, self.attn_dim//self.n_head)).transpose(1, 2)
        scores= torch.matmul(Q, torch.transpose(K, 3, 2))/(self.attn_dim//self.n_head)**0.5
        
        if non_lin:
            scores= torch.softmax(scores, dim = 3)
        
        Y1 = torch.matmul(scores, V).transpose(1, 2).reshape(N, self.seq_len, self.attn_dim)
        out_att = self.proj(Y1)

        return out_att
    
    
class sort_limited():
    def __init__(self, leng, ran, num_train = 50000):
        self.len = leng
        self.ran= ran
        self.num_train = num_train

    def get_leveled(self, num_swaps, b):
        sorted_seq = np.sort(np.random.choice(self.ran, [b, self.len]))
        seq = np.copy(sorted_seq)
        for _ in np.arange(num_swaps):
            i = np.random.randint(self.len-1, size= (b,))
            t= seq[np.arange(b), i]
            seq[np.arange(b), i] = seq[np.arange(b), i+1]
            seq[np.arange(b), i+1] = t
        
        indices = np.argsort(seq)
        seq = np.eye(self.ran)[seq]

        return seq, indices
    
    def get_test(self, b):
        seq= np.random.choice(self.ran, [b, self.len])
        sorted_seq = np.argsort(np.argsort(seq, axis = 1))
        seq = np.eye(self.ran)[seq]
        
        return seq, sorted_seq
    
    def get_train(self, b):
        train_list = np.random.choice(self.num_train, b, replace= True)
        seq = []
        for i in train_list:
            np.random.seed(i)
            seq.append(np.random.choice(self.ran, (self.len)))
        np.random.seed()
        seq = np.stack(seq, axis = 0)
        sorted_seq = np.argsort(np.argsort(seq, axis = 1))
        seq = np.eye(self.ran)[seq]
        
        return seq, sorted_seq

In [ ]:
lr= 2e-4
tran = MultiHeadAttention_wOhead(20, 200, 20, 32, 1)
tran.to(device= device)
gener= sort_limited(20, 200, 3000)
opt = torch.optim.Adam(tran.parameters(), lr= lr)
loss_fn = nn.CrossEntropyLoss()
print("learning rate= ", (lr))
trainer(tran, gener, 250000, opt, loss_fn, verbose= True, non_lin= False)

learning rate=  0.0002


Progress Bar:   0%|                                                                                                                                                       | 15/250000 [00:00<2:52:05, 24.21it/s]

step 0: loss = 2.995725, train_acc= 0.050425, test_acc= 0.051080


Progress Bar:   2%|███                                                                                                                                                    | 5027/250000 [00:32<58:22, 69.94it/s]

step 5000: loss = 1.552382, train_acc= 0.384120, test_acc= 0.383575


...

Progress Bar:  92%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏           | 230033/250000 [21:04<03:03, 108.68it/s]

step 230000: loss = 0.199771, train_acc= 0.930500, test_acc= 0.927550


Progress Bar:  94%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████         | 235039/250000 [21:30<02:39, 93.76it/s]

step 235000: loss = 0.291110, train_acc= 0.929575, test_acc= 0.927300


Progress Bar:  95%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋       | 237706/250000 [21:43<00:58, 208.78it/s]

In [8]:
class sort_limited_proj():
    def __init__(self, leng, ran, num_train = 50000):
        self.len = leng
        self.ran= ran
        self.num_train = num_train
        self.proj = np.random.randn(ran, ran) / np.sqrt(ran)

    def get_leveled(self, num_swaps, b):
        sorted_seq = np.sort(np.random.choice(self.ran, [b, self.len]))
        seq = np.copy(sorted_seq)
        for _ in np.arange(num_swaps):
            i = np.random.randint(self.len-1, size= (b,))
            t= seq[np.arange(b), i]
            seq[np.arange(b), i] = seq[np.arange(b), i+1]
            seq[np.arange(b), i+1] = t
        seq = np.eye(self.ran)[seq] @ self.proj

        return seq, sorted_seq
    
    
    def get_leveled_train(self, num_swaps, b):
        sorted_seq = np.sort(np.random.choice(self.ran, [b, self.len]))
        seq = np.copy(sorted_seq)
        for _ in np.arange(num_swaps):
            i = np.random.randint(self.len-1, size= (b,))
            t= seq[np.arange(b), i]
            seq[np.arange(b), i] = seq[np.arange(b), i+1]
            seq[np.arange(b), i+1] = t
        seq = np.eye(self.ran)[seq] @ self.proj

        return seq, sorted_seq
    
    
    def get_test(self, b):
        seq= np.random.choice(self.ran, [b, self.len])
        sorted_seq = np.sort(seq)
        seq = np.eye(self.ran)[seq] @ self.proj
        
        return seq, sorted_seq
    
    
    def get_train(self, b):
        train_list = np.random.choice(self.num_train, b, replace= True)
        seq = []
        for i in train_list:
            np.random.seed(i)
            seq.append(np.random.choice(self.ran, (self.len)))
        np.random.seed()
        seq = np.stack(seq, axis = 0)
        sorted_seq = np.sort(seq)
        seq = np.eye(self.ran)[seq] @ self.proj
        
        return seq, sorted_seq 
            

In [31]:
for lr in np.logspace(-5, -2, 10):
    tran = MultiHeadAttention(20, 200, 200, 256, 256, 100, 32)
    tran.to(device= device)
    gener= sort_limited_proj(20, 200, 1000)
    opt = torch.optim.Adam(tran.parameters(), lr= lr)
    loss_fn = nn.CrossEntropyLoss()
    print("learning rate= ", (lr))
    trainer(tran, gener, 5000, opt, loss_fn, verbose= False, non_lin= True)

learning rate=  1e-05


Progress Bar: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5000/5000 [00:18<00:00, 264.90it/s]


final loss= 4.639436
learning rate=  2.1544346900318823e-05


Progress Bar: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5000/5000 [00:18<00:00, 277.23it/s]


final loss= 3.923155
learning rate=  4.641588833612782e-05


...

final loss= 0.911591
learning rate=  0.004641588833612777


Progress Bar: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5000/5000 [00:17<00:00, 282.54it/s]


final loss= 11.803789
learning rate=  0.01


Progress Bar: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5000/5000 [00:18<00:00, 273.29it/s]

final loss= 20.750059


In [6]:
lr= 5e-4
tran = MultiHeadAttention(20, 200, 200, 256, 256, 100, 32)
tran.to(device= device)
gener= sort_limited_proj(20, 200, 1000)
opt = torch.optim.Adam(tran.parameters(), lr= lr)
loss_fn = nn.CrossEntropyLoss()
print("learning rate= ", (lr))
trainer(tran, gener, 250000, opt, loss_fn, verbose= True, non_lin= True)

learning rate=  0.0005


Progress Bar:   0%|                                                                                                                                                       | 30/250000 [00:00<1:39:09, 42.01it/s]

step 0: loss = 5.451015, train_acc= 0.004940, test_acc= 0.006095


Progress Bar:   2%|███                                                                                                                                                    | 5026/250000 [00:19<46:09, 88.46it/s]

step 5000: loss = 3.327105, train_acc= 0.266605, test_acc= 0.154195


...

step 240000: loss = 0.002054, train_acc= 0.989105, test_acc= 0.253020


Progress Bar:  98%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████   | 245044/250000 [15:19<00:49, 100.36it/s]

step 245000: loss = 0.002617, train_acc= 0.987595, test_acc= 0.253050


Progress Bar: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 250000/250000 [15:36<00:00, 266.85it/s]

final loss= 0.000006


In [11]:
lr= 5e-4
tran = MultiHeadAttention(20, 200, 200, 256, 256, 100, 32)
tran.to(device= device)
gener= sort_limited_proj(20, 200, 5000)
opt = torch.optim.Adam(tran.parameters(), lr= lr)
loss_fn = nn.CrossEntropyLoss()
print("learning rate= ", (lr))
trainer(tran, gener, 250000, opt, loss_fn, verbose= True, non_lin= True)

learning rate=  0.0005


Progress Bar:   0%|                                                                                                                                                       | 26/250000 [00:00<1:49:16, 38.13it/s]

step 0: loss = 5.498064, train_acc= 0.005120, test_acc= 0.004965


Progress Bar:   2%|███                                                                                                                                                   | 5040/250000 [00:20<40:07, 101.77it/s]

step 5000: loss = 3.594720, train_acc= 0.129020, test_acc= 0.115685


...

step 50000: loss = 1.143079, train_acc= 0.797350, test_acc= 0.687170


Progress Bar:  22%|█████████████████████████████████                                                                                                                     | 55024/250000 [03:30<37:02, 87.74it/s]

step 55000: loss = 0.351584, train_acc= 0.812335, test_acc= 0.698990


Progress Bar:  24%|███████████████████████████████████▊                                                                                                                 | 60000/250000 [03:49<12:05, 262.01it/s]


KeyboardInterrupt: 

In [4]:
lr= 5e-4
tran = MultiHeadAttention(20, 200, 200, 256, 256, 100, 32)
tran.to(device= device)
gener= sort_limited_proj(20, 200, 3000)
opt = torch.optim.Adam(tran.parameters(), lr= lr)
loss_fn = nn.CrossEntropyLoss()
print("learning rate= ", (lr))
trainer(tran, gener, 250000, opt, loss_fn, verbose= True, non_lin= True)

NameError: name 'MultiHeadAttention' is not defined

In [ ]:
class sort_limited_proj_curr():
    def __init__(self, leng, ran, curriculum_levels, curriculum_nums):
        self.len = leng
        self.ran= ran
        self.proj = np.random.randn(ran, ran) / np.sqrt(ran)
            
        self.levels = curriculum_levels
        self.nums = curriculum_nums
        self.indices = {}
        assert len(self.levels) == len(self.nums), "lengths don't match"
        
        index = np.random.choice(50000, np.sum(self.nums), replace = False)
        counter= 0

        for j in range(len(self.nums)):
            self.indices{"%i"%(levels[j])} = index[counter:counter+self.nums[j]]
    
    def get_leveled_train(self, num_swaps, b):
        train_set = self.indices{"%i"%(num_swaps)}
        train_list = np.random.choice(train_set, b, replace= True)
        seqs = []
        
        for i in train_list:
            np.random.seed(i)
            sorted_seq = np.sort(np.random.choice(self.ran, (self.len)))
            seq = np.copy(sorted_seq)
            for _ in np.arange(num_swaps):
                i = np.random.randint(self.len-1)
                t= seq[i]
                seq[i] = seq[i+1]
                seq[i+1] = t
            seqs.append(seq)
            
        seqs = np.stack(seqs, axis = 0)
        sorted_seqs = np.sort(seqs)
        seqs = np.eye(self.ran)[seqs] @ self.proj

        return seqs, sorted_seqs
    
    
    def get_leveled_test(self, num_swaps, b):
        sorted_seq = np.sort(np.random.choice(self.ran, [b, self.len]))
        seq = np.copy(sorted_seq)
        for _ in np.arange(num_swaps):
            i = np.random.randint(self.len-1, size= (b,))
            t= seq[np.arange(b), i]
            seq[np.arange(b), i] = seq[np.arange(b), i+1]
            seq[np.arange(b), i+1] = t
        seq = np.eye(self.ran)[seq] @ self.proj

        return seq, sorted_seq
    
    
    def get_test(self, b):
        seq= np.random.choice(self.ran, [b, self.len])
        sorted_seq = np.sort(seq)
        seq = np.eye(self.ran)[seq] @ self.proj
        
        return seq, sorted_seq
    

def trainer_curr(model, gener, iters, opt, loss_fn, swap_plan, verbose= True, non_lin = False, device= "cuda:1"):
    for step, swap_num in enumerate(swap_plan):
        for i in tqdm(range(iters), desc = 'Progress Bar'):
            (X, Y) = gener.get_leveled_train(64)
            preds= model(torch.tensor(X, device = device, dtype= torch.float32), non_lin = non_lin)
            loss= loss_fn(preds.transpose(1, 2), torch.tensor(Y, device = device))

            loss.backward()
            opt.step()
            opt.zero_grad()


            if i%5000 == 0 and verbose:  
                with torch.no_grad():
                    (X_train, Y_train) = gener.get_train(10000)
                    preds_train= torch.argmax(model(torch.tensor(X_train, device = device, dtype= torch.float32), non_lin = non_lin), dim = 2)
                    train_acc = (torch.sum(1*(preds_train == torch.tensor(Y_train, device = device)))/2e5).detach().cpu().numpy()

                    (X_valid, Y_valid) = gener.get_test(10000)
                    preds_valid= torch.argmax(model(torch.tensor(X_valid, device = device, dtype= torch.float32), non_lin = non_lin), dim = 2)
                    test_acc = (torch.sum(1*(preds_valid == torch.tensor(Y_valid, device = device)))/2e5).detach().cpu().numpy()

                print("step %i: loss = %f, train_acc= %f, test_acc= %f"%(i, loss, train_acc, test_acc))
                torch.save({"model": model.state_dict(),
                            "opt": opt.state_dict(),
                            "num_train": i}, "./Sorting_limited.tar")
        print("final loss= %f"%(loss.detach().cpu().numpy()))
